# Mejores matrices Origen-Destino

En el primer notebook de accesibiliodad implementamos nuestros modelos utilizando la distancia euclidiana. Esto es, desde luego, relativamente sencillo de calcular, sin embargo claramente no es lo ideal. El siguiente paso lógico es utilizar la conectividad sobre la red de calles.

Hay muchísimas formas de obtener matrices de costo utilizando la red de calles, desde luego todas ellas van a tener algunas complicaciones, se necesitan más datos (la red de calles, dah) y es computacionalmente intensivo. En la documentación de access tenemos una [lista](https://access.readthedocs.io/en/latest/resources.html) de diferentes herramientas para crear estas matrices.

Nosotros vamos a usar en este ejemplo [OSRM](http://project-osrm.org/) que es un motor de rutas que utiliza datos de OSM. Para esto tenemos una instalación de OSRM en un servidor de CentroGeo. Para acceder a ese servidor tienen que sacar una cuenta con su correo institucional en [tailscale](tailscale.com). Ya que tengan su cuenta necesitan seguir las [instrucciones](https://tailscale.com/download/windows) para instalar el cliente de tailscale en su plataforma. Ya con el cliente configurado pueden acceder a la interfaz gráfica aquí:

http://100.67.66.19:9966/

En lo que sigue de este notebook voy a usar datalab en lugar de la ip para escribir menos (para que eso funcione necesitan editar el hosts file de sus compus o el equivalente en windows).

OSRM es un servidor que nos provee un API para hacer algunos cálculos de rutas. La documentación completa del API la pueden ver [aquí](http://project-osrm.org/docs/v5.24.0/api/#). Para hacer peticiones al API desde Python, vamos a usar la librería [requests](https://docs.python-requests.org/en/master/). Para empezar, hagamos una petición pidiendo la ruta y las instrucciones entre dos puntos. 

## Ejemplo de ruta

In [ ]:
import requests
r = requests.get('http://100.67.66.19:5000/route/v1/driving/-99.14428918256026,19.351424122115958;-99.16261403345554,19.35211246418011?steps=true')
r.json()

Como ven, es muy fácil de usar, desde luego la salida es un poco complicada, pero en el fondo es sólo un json, entonces es sencillo de parsear y utilizar como mejor nos convenga. 


## Ejemplo de matriz

Como nuestro problema básico ahorita no es obtener rutas sino calcular matrices origen destino, nos vamos a concentrar en ello en lugar de dar un paseo completo por el API de OSRM.

El único servicio que vamos a necesitar el [table-service](http://project-osrm.org/docs/v5.24.0/api/#table-service) que toma dos listas de coordenadas y nos regresa las distancias (sobre la red) y costos entre todos los pares de coordenadas. Justo lo que queremos.

Para empezar a trabajar con nuestros datos, leamos los datos de centroides de áreas verdes y agebs.

In [ ]:
import pandas as pd
import geopandas as gpd
agebs_zmvm_centroides = gpd.read_file('./datos/agebs_zmvm_centroides.gpkg', layer='agebs_zmvm_centroides')
escuelas = gpd.read_file("./datos/escuelas_zmvm_centroides.gpkg", layer='escuelas_zmvm_centroides')

Tomemos sólo los primeros registros de cada una, ahorita sólo queremos entender la estructura que nos regresa el API y cómo crear la matriz que queremos

In [ ]:
# Tomamos los primeros 5 registros de cada dataframe
origenes = agebs_zmvm_centroides.head()
destinos = escuelas.head()

Ahora tenemos que manufacturar el string de la petición al API. Viendo la documentación, el servicio nos regresa la distancia (o el costo) entre todos los pares de coordenadas, o bien, podemos identificar un par de coordenadas como el origen y entonces nos regrasa la distancia desde ese punto a todos los demás. Por la estructura de nuestros datos, lo más fácil es tomar el primer punto de origen y calcular las distancias a todos los demás y después iterar sobre los origenes.


Una petición del tipo que queremos se ve así:

````bash
curl 'http://router.project-osrm.org/table/v1/driving/13.388860,52.517037;13.397634,52.529407;13.428555,52.523219?sources=0'
````

El primer par de coordenadas es el origen y el resto son los destinos. 


### Strings de las peticiones

Ahora necesitamos transformar nuestros datos en los strings que necesitamos para hacer las peticiones al API. El primer paso es hacer una lista con las coordenadas de los destinos

In [ ]:
# Hacemos una lista de las latitudes y longitudes
ys = destinos.geometry.to_crs(4326).y.to_list()
xs = destinos.geometry.to_crs(4326).x.to_list()
# zipeamos la lista
pares = list(zip(ys,xs))
pares

Transformamos la lista en el string que necesitamos

In [ ]:
s = []
for p in pares:
    a = str(p[1]) + "," + str(p[0])
    s.append(a)
destinos_str = ";".join(s)
destinos_str

Hacemos el string del origen

In [ ]:
origen_str = str(origenes.to_crs(4326).iloc[0].geometry.x) + ',' + str(origenes.to_crs(4326).iloc[0].geometry.y)
origen_str

Hacemos el string completo del request al API

In [ ]:
base_url = 'http://100.67.66.19:5000/table/v1/driving/'
full_request = base_url + origen_str + ';' + destinos_str + '?sources=0'
full_request

### Petición
Ahora podemos hacer la petición

In [ ]:
import requests
r = requests.get(full_request)
r.json()

De acuerdo a la documentación de OSRM, lo que nos regresa el API es un json que en la entrada `durations` tiene las duraciones de los viajes a cada uno de los destinos conservando el índice en el que los mandamos, las demás entradas contienen información adicional que en este caso no vamos a utilizar. Entonces, lo que necesitamos hacer es simplemete extraer la lista de durations

In [ ]:
r.json()['durations'][0]

El primer elemento de la lista no lo queremos, es la duración entre el origen y el origen mismo, los demás son los buenos. Ahora necesitamos estructurar los datos de forma que podamos obtener el dataframe de costos como lo necesitamos para `access`. Fíjense que tenemos el id del origen en `origenes.iloc[0].CVEGEO` y los ids de los destinos en el dataframe correspondiente, entonces es relativamente fácil hacur un nuevo dataframe con origen, destino y costo

In [ ]:
costos = destinos.copy()[['id_escuela']].rename({'id_escuela':'destino'}, axis=1)
costos['costo'] = r.json()['durations'][0][1:]
costos['origen'] = origenes.iloc[0].CVEGEO
costos

## Todo en una función
Perfecto, tenemos el pedazo de la matriz que corresponde al primer origen, ahora ya sólo necesitamos iterar sobre todos los orígenes y concatenar el resultado en un sólo dataframe.

Para hacer todo más fácil, lo que más nos conviene es empaquetar todo el flujo que hemos hecho en una sóla función

In [ ]:
def get_od_matrix(origenes, destinos):
    base_url = 'http://100.67.66.19:5000/table/v1/driving/'
    # primero hacemos todo el string de los destinos
    ys = destinos.geometry.to_crs(4326).y.to_list()
    xs = destinos.geometry.to_crs(4326).x.to_list()
    pares = zip(ys,xs)
    s = []
    for p in pares:
        a = str(p[1]) + "," + str(p[0])
        s.append(a)
    destinos_str = ";".join(s)
    #para cada origen hacemos la patición y guardamos el resultado
    origenes = origenes.to_crs(4326)
    costos = [] # acá vamos a ir guardando los resultados para cada origen
    for _, row in origenes.iterrows():
        origen_str = str(row.geometry.x) + ',' + str(row.geometry.y)
        req = base_url + origen_str + ';' + destinos_str + '?sources=0'
        r = requests.get(req)
        resultado = r.json()
        if resultado['code'] == 'Ok':
            c = destinos.copy()[['id_escuela']].rename({'id_escuela':'destino'}, axis=1)
            c['costo'] = resultado['durations'][0][1:]
            c['origen'] = row.CVEGEO
            costos.append(c)
        else:
            print(f"something went wrong with origin {row.CVEGEO}")
    costos = pd.concat(costos).reset_index().drop(columns=['index'])
    return costos
        
    

Probamos nuestra función con los datos chiquitos

In [ ]:
costos = get_od_matrix(origenes, destinos)
costos

Ya con nuestra función, podemos ahora sí hacer la matriz completa, sólo que eso evidentemente va a tardar mucho. Una forma de ganarle tiempo es correr el loop de la función en paralelo

In [ ]:
from joblib import Parallel, delayed
def parallel_get_od_matrix(origenes, destinos, njobs=2):
    def process_row(_, row):
            origen_str = str(row.geometry.x) + ',' + str(row.geometry.y)
            req = base_url + origen_str + ';' + destinos_str + '?sources=0'
            r = requests.get(req)
            resultado = r.json()
            if resultado['code'] == 'Ok':
                c = destinos.copy()[['id_escuela']].rename({'id_escuela':'destino'}, axis=1)
                c['costo'] = resultado['durations'][0][1:]
                c['origen'] = row.CVEGEO
            else:
                print(f"something went wrong with origin {row.CVEGEO}")
            return c
    base_url = 'http://100.67.66.19:5000/table/v1/driving/'
    # primero hacemos todo el string de los destinos
    ys = destinos.geometry.to_crs(4326).y.to_list()
    xs = destinos.geometry.to_crs(4326).x.to_list()
    pares = zip(ys,xs)
    s = []
    for p in pares:
        a = str(p[1]) + "," + str(p[0])
        s.append(a)
    destinos_str = ";".join(s)
    #para cada origen hacemos la patición y guardamos el resultado
    origenes = origenes.to_crs(4326)
    costos = [] # acá vamos a ir guardando los resultados para cada origen
    costos = Parallel(n_jobs=njobs)(delayed(process_row)(_,row) for _, row in origenes.iterrows())
    costos = pd.concat(costos).reset_index().drop(columns=['index'])
    return costos

Probamos nuestra nueva función y comparamos los tiempos de ejecución (noten que estamos usando sólo dos trabajos simultaneamente)

Ganamos un 30%!!!!

Con esta función podemos ya correr el algoritmo para la base de datos completa. Se va a tardar muchísimo, no lo intenten en casa!!!! 

**NOTA:** Pueden jugar un poco con el valor de njobs, pero recuerden que debe ser menor al número de hilos de su procesador

## Matriz completa

Pues ahora ya estamos listos para hacer la matriz inmensa

In [ ]:
matriz_completa = parallel_get_od_matrix(agebs_zmvm_centroides, escuelas)
matriz_completa.head(20)

La guardamos como pickle

In [ ]:
matriz_completa.to_csv("./datos/matriz_escuelas_od_walking.csv". index=False)
matriz_completa.to_file("./datos/matriz_escuelas_od_walking.gpkg", layer='matriz_escuelas_od_walking', driver='GPKG')